In [1]:
import scipy.io
import os 
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import scipy 
import scipy.stats as stats
from scipy.stats import sem 
import json
import random
import copy

In [4]:
homedir = '/Users/prachi/Documents/depth_duration/depth_discrimination/'

In [40]:
# load 48x48 latin square from MATLAB randls function
randls24 = scipy.io.loadmat(homedir + 'randls24.mat')['ans']
randls94 = scipy.io.loadmat(homedir + 'randls94.mat')['ans']

randls94.shape, randls94[0]

((94, 94),
 array([86,  5, 48, 41, 81, 90, 17, 66, 62, 67, 89, 51, 35, 38, 46, 15, 64,
        78, 13, 69,  8, 49, 76,  4, 14, 75, 57, 55, 73, 18, 28, 68, 72, 83,
         6, 31, 54, 11, 60, 12, 36, 21,  9, 80, 24, 50, 47, 94, 32, 52, 58,
        29, 85, 87, 44, 74, 91, 10, 39, 65,  7, 19, 45, 82, 42, 77, 59, 33,
        56, 40, 79,  2, 22, 92, 25,  1, 37, 84, 23, 63, 30, 71, 88,  3, 34,
        70, 16, 20, 53, 27, 43, 61, 26, 93], dtype=uint8))

In [41]:
# load an existing balanced VE sequence and seperate into 4 blocks
discrim_seq_path = '/Users/prachi/Documents/depth_duration/depth_discrimination/discrimination_sequences/january2022/j13/j13.json'

with open(discrim_seq_path, 'r') as f:
    discrim_seq = json.load(f)


In [42]:
discrim_seq_without_catch = []
catch = []
for trial in discrim_seq:
    if trial['sequence'] != 'catch_trial':
        discrim_seq_without_catch.append(trial)
    else:
        catch.append(trial)

len(discrim_seq_without_catch), len(catch)

(94, 8)

In [15]:
def randls_to_sequence(randls_row, block):
    '''
    Uses indeces from the randls matrix row to build sequence
    
    Args:
        randls_row - one row of the randls matrix
        block - one block out of the four in the VE sequence
    Returns:
        VE trials in sequence
    '''
    sequence = []
    for elem in randls_row:
        if elem == len(randls_row):
            idx = 0
        else:
            idx = elem
        sequence.append(block[idx])
    return sequence 

def main_randls_blocks(randls, block):
    '''
    Converts each randls row into a sequence 
    
    Args:
        randls - randls matrix from MATLAB
        block - one block out of the four in the VE sequence
    Returns:
        All sequences from the randls matrix in trial sequence format
    '''
    block_sequences = []
    for i in range(randls.shape[0]):
        row = randls[i]
        seq = randls_to_sequence(row, block)
        block_sequences.append(seq)
    return block_sequences

In [43]:
full_randls_seqs = main_randls_blocks(randls94, discrim_seq_without_catch)


In [44]:
len(full_randls_seqs), len(full_randls_seqs[0])

(94, 94)

In [71]:
# shuffle the order of the sequences
random.shuffle(full_randls_seqs)


In [73]:
# build the blocks together
sequences47 = []
# pick 47/94 sequences
for i in range(47):
    random.shuffle(catch)
    fseq = full_randls_seqs[i]
    catch_locs = [random.randint(0, 11), random.randint(12, 23), random.randint(24, 35), random.randint(36, 47),
                  random.randint(48, 59), random.randint(60, 71), random.randint(72, 83), random.randint(84, 93)]
    for i in range(len(catch)):
        fseq.insert(catch_locs[i], catch[i])
    
    sequences47.append(fseq)
    
len(sequences47), len(sequences47[0]), sequences47[0][0]

(47,
 118,
 {'sequence': 'j13',
  'duration': 1000,
  'depth_0': 1.5043333333333333,
  'depth_1': 2.074,
  'image_path_target_0': 'depth_discrimination_stimuli/002279_2014-06-28_18-55-32_260595134347_rgbf000040-resize_5/002279_2014-06-28_18-55-32_260595134347_rgbf000040-resize_5-target.png',
  'image_path_target_1': 'depth_discrimination_stimuli/000635_2014-06-08_16-38-42_260595134347_rgbf000106-resize_1/000635_2014-06-08_16-38-42_260595134347_rgbf000106-resize_1-target.png',
  'mask_path': 'masks/mask_55.jpg',
  'fixation_path': 'fixation.jpg'})

In [74]:
# set all trial durations to 250 ms
for seq in sequences47:
    for trial in seq:
        trial['duration'] = 250

sequences47[0][0]

{'sequence': 'j13',
 'duration': 250,
 'depth_0': 1.5043333333333333,
 'depth_1': 2.074,
 'image_path_target_0': 'depth_discrimination_stimuli/002279_2014-06-28_18-55-32_260595134347_rgbf000040-resize_5/002279_2014-06-28_18-55-32_260595134347_rgbf000040-resize_5-target.png',
 'image_path_target_1': 'depth_discrimination_stimuli/000635_2014-06-08_16-38-42_260595134347_rgbf000106-resize_1/000635_2014-06-08_16-38-42_260595134347_rgbf000106-resize_1-target.png',
 'mask_path': 'masks/mask_55.jpg',
 'fixation_path': 'fixation.jpg'}

In [75]:
# deepcopy of sequences - prevents alterations 
sequences47_250ms = copy.deepcopy(sequences47)

In [76]:
# set all trial durations to 1000 ms
for seq in sequences47:
    for trial in seq:
        trial['duration'] = 1000

sequences47[0][0]

{'sequence': 'j13',
 'duration': 1000,
 'depth_0': 1.5043333333333333,
 'depth_1': 2.074,
 'image_path_target_0': 'depth_discrimination_stimuli/002279_2014-06-28_18-55-32_260595134347_rgbf000040-resize_5/002279_2014-06-28_18-55-32_260595134347_rgbf000040-resize_5-target.png',
 'image_path_target_1': 'depth_discrimination_stimuli/000635_2014-06-08_16-38-42_260595134347_rgbf000106-resize_1/000635_2014-06-08_16-38-42_260595134347_rgbf000106-resize_1-target.png',
 'mask_path': 'masks/mask_55.jpg',
 'fixation_path': 'fixation.jpg'}

In [77]:
# deepcopy of sequences - prevents alterations 
sequences47_1000ms = copy.deepcopy(sequences47)

In [78]:
sequences47_250ms[0][0]['duration'], sequences47_1000ms[0][0]['duration']

(250, 1000)

In [79]:
dest_250ms = '/Users/prachi/Documents/depth_duration/depth_discrimination/discrimination_sequences/randls_sequences/250ms/'
dest_1000ms = '/Users/prachi/Documents/depth_duration/depth_discrimination/discrimination_sequences/randls_sequences/1000ms/'

In [80]:
# save sequences as jsons 
num = 0
for seq in sequences47_250ms:
    with open(dest_250ms + 'randls_' + str(num)+ '.json', 'w') as outfile:
        json.dump(seq, outfile)
        num += 1

In [81]:
# save sequences as jsons 
num = 0
for seq in sequences47_1000ms:
    with open(dest_1000ms + 'randls_' + str(num)+ '.json', 'w') as outfile:
        json.dump(seq, outfile)
        num += 1

# edit code so that it flips the order of the images and the depth tagging appropriately

In [84]:
import os 

def load_sequence(jsonpath):
    return json.load(open(jsonpath))

def flip_image_order(previous_seq):
    flipped = previous_seq
    for i in range(len(previous_seq)):
        og_depth0 = previous_seq[i]['depth_0']
        og_depth1 = previous_seq[i]['depth_1']
        og_path0 = previous_seq[i]['image_path_target_0']
        og_path1 = previous_seq[i]['image_path_target_1']
        
        flipped[i]['depth_0'] = og_depth1
        flipped[i]['depth_1'] = og_depth0
        flipped[i]['image_path_target_0'] = og_path1
        flipped[i]['image_path_target_1'] = og_path0
        
    return flipped

def create_flipped_seqs(jsonpath, exit, name):
    master = load_sequence(jsonpath)
    
    r = flip_image_order(master)
    r_path = exit + '/' + name + '_rotated.json' # duration rotated sequence
    #creates json file for the sequence 
    with open(r_path, 'w') as f:
        json.dump(r , f)

        
def main_seq_flips(json_folderpath, exit):
    """
    Create rotated sequence for each sequence in the folder
    """
    for file in os.listdir(json_folderpath):
        name = file.split(".")[0]
        jsonpath = json_folderpath + "/" + file
        try:
            create_flipped_seqs(jsonpath, exit, name)
        except:
            print("Failed to create json flips for: ", file)

In [85]:
destination = '/Users/prachi/Documents/depth_duration/depth_discrimination/discrimination_sequences/randls_sequences/250ms'

main_seq_flips(destination,destination)

Failed to create json flips for:  .DS_Store
Failed to create json flips for:  .ipynb_checkpoints


In [86]:
destination2 = '/Users/prachi/Documents/depth_duration/depth_discrimination/discrimination_sequences/randls_sequences/1000ms'

main_seq_flips(destination2,destination2)

Failed to create json flips for:  .DS_Store
